## Getting Started

I have started a course on probability from [MIT Open courseware](https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-436j-fundamentals-of-probability-fall-2018/)

In an attempt to solidify my learning and also get started on my blog I will try to code interesting problems/topics that I come across.

Starting with the [Monty Hall problem](https://en.wikipedia.org/wiki/Monty_Hall_problem).

In this notebook, I will attempt to:
- Create a Simulation of a single trial of the Monty Hall Problem (with the option to switch)
- Run the simulation multile number of times (with switching, without switching and random switchnig) and explore the results
- Figure out if I can make the simulation faster

To get started, lets define a function that takes in three inputs (Prize door #, Selected door #, Switch) and returns the result of the experiment (win or loss).

Additionally in this case, a win with switching is a loss without switching (and vice versa)

In [1]:
# Returns if the trial results in a win
def MontyTrial(win_door, selected_door, switch = False):
    if (win_door == selected_door and not switch):
        return 1
    elif (win_door == selected_door and switch):
        return 0
    elif (win_door != selected_door and switch):
        return 1
    elif (win_door != selected_door and not switch):
        return 0

### Simulating the trials

Now that we have a function that can give us the outcome of an individual trial, lets see if we can run multiple trials and find out what happens.

I will use a Numpy array to generate random winning door numbers and selected door numbers.

In [2]:
# Import numpy and datetime. I will use current UTC as the seed for Numpy's random number generator
import numpy as np
from datetime import datetime

seed = datetime.utcnow().timestamp()

#Testing generation of 10 random ints between 1 and 3

print("Seed: ", int(seed))
np.random.seed(int(seed))

print("10 Random integers between 1 & 3: ", np.random.randint(1,4,10))



Seed:  1611073640
10 Random integers between 1 & 3:  [2 2 2 3 3 2 1 2 3 1]


Having set that up, I will now generate two arrays of size 1000 each. One would be the winning door number and the other would be the selected door number.

Then we can feed in the results to the MontyTrial function and evaluate the results

Edit: Going to convert this into a function so that I can time the results

In [3]:
#This function runs Monty Hall trials for the given number of trials. 
#The supress variable will stop any output from the function for timing purposes
def MontySimulation(trials, supress=False):
    if (supress == False):
        print("Simulating ",trials," trials")
    Winning_Door = np.random.randint(1,4,trials)
    Selected_Door = np.random.randint(1,4,trials)

    Result = 0

    #Running the trial where the contestant never switches
    for i in range(0, len(Winning_Door)):
        Result += MontyTrial(Winning_Door[i], Selected_Door[i], False)

    Result = Result/trials
    if (supress == False):
        print("Probability of winning with no switching: ", Result)
        print("Probability of winning with always switching: ", 1 - Result)
    
MontySimulation(100000)

Simulating  100000  trials
Probability of winning with no switching:  0.33394
Probability of winning with always switching:  0.66606


Looking at the results above, the probability of winning is ~67% or 2/3rd of the time if an always switching strategy is adopted.

However I think this implementation is not optimal and there is an opportunity to speed it up. To start, I will time 1 million trials.

In [4]:
%timeit MontySimulation(1000000, True)

817 ms ± 145 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


With the current logic, it takes about 1 second to run 1 million MontyHall trials. There is obviously room for improvement. To start, lets see if we can simplyfy the Monty Trial function

In [5]:
#In the v2 version of this function, I will try to reduce the for conditionals and run a trial to see if that improves performance
def MontyTrialv2(win_door, selected_door, switch = False):
    result = 0 if win_door == selected_door else 1
    return result == switch    

Came down to a bit of a wierd logic, but this is what the function is doing:

1. If Winning door and selected door are the same then set result to 0, else set it to 1
2. If there is no switching, then the variable switch is false (0) and so result would be equal to switch (and a win)
3. If result is one and no switching, then the return statement returns as false (1 == 0)
4. With switching (variable switch set to true) a zero resutls returns a false (since 0 != 1)
5. Finally, with switching set to true a 1 results returns True (since 1 == 1)

Lets plug this new function into a simulator and see if this speeds up the code

In [6]:
#V2 simulation fucntion uses the Monty V2 to generate results
def MontySimulationv2(trials, supress=False):
    if (supress == False):
        print("Simulating ",trials," trials in v2")
    Winning_Door = np.random.randint(1,4,trials)
    Selected_Door = np.random.randint(1,4,trials)

    Result = 0

    #Running the trial where the contestant never switches
    for i in range(0, len(Winning_Door)):
        Result += MontyTrialv2(Winning_Door[i], Selected_Door[i], False)

    Result = Result/trials
    if (supress == False):
        print("Probability of winning with no switching: ", Result)
        print("Probability of winning with always switching: ", 1 - Result)
    
MontySimulation(100000)
MontySimulationv2(100000)

Simulating  100000  trials
Probability of winning with no switching:  0.33181
Probability of winning with always switching:  0.6681900000000001
Simulating  100000  trials in v2
Probability of winning with no switching:  0.33161
Probability of winning with always switching:  0.66839


In [7]:
print("Timing v1 Simulation on 1 million trials")
%timeit MontySimulation(1000000, True)
print("Timing v2 Simulation on 1 million trials")
%timeit MontySimulationv2(1000000, True)

Timing v1 Simulation on 1 million trials
735 ms ± 16.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Timing v2 Simulation on 1 million trials
659 ms ± 25.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Not a measurable impact. It could be the different arrays generated in each function (unlikely!!) so lets just time the main trail function itself

In [8]:
print("Timing v1 trial function")
%timeit MontyTrial(1,2,True)
print("Timing v2 trial function")
%timeit MontyTrialv2(1,2,True)

Timing v1 trial function
216 ns ± 17.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
Timing v2 trial function
178 ns ± 23.3 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Marginal impact. So lets try a completely different approach. Building this simulator, i have realised:

- There is a mathematical function that can return the result of the simulation (thus there will ne no need for conditionals)
- A win in the non-switching trial would be a loss in the switching trial (and vice versa) since there are only two doors left to choose from

So using Numpy there could be a faster implementation.

In [11]:
#Numpy based implementation
def MontySimulationv3(trials, supress=False):
    if (supress == False):
        print("Simulating ",trials," trials in v3 (Numpy Version)")
    #Generate a winning and a trial array and compare. A true in this array results in a win in case of no switching
    Result = np.random.randint(1,4,trials) == np.random.randint(1,4,trials)
    
    #If Winning door and the selected doors are the same and there is no switching, then its a win
    Noswitch_win = Result.sum()/trials
    
    if (supress == False):
        print("Probability of winning with no switching: ", Noswitch_win)
        #Winning with switching is 1 - probability of winning without switching
        print("Probability of winning with switching: ", 1 - Noswitch_win)
    

In [12]:
MontySimulationv3(1000000)

Simulating  1000000  trials in v3 (Numpy Version)
Probability of winning with no switching:  0.333419
Probability of winning with switching:  0.666581


The final version of the simulator uses Numpy only to run trials. All logic is boiled down to a simple comparison of the winning door array and selected door array. Lets find out if that speeds up the model

In [13]:
print("Timing v1 Simulation on 1 million trials")
%timeit MontySimulation(1000000, True)
print("Timing v2 Simulation on 1 million trials")
%timeit MontySimulationv2(1000000, True)
print("Timing v3 Simulation on 1 million trials")
%timeit MontySimulationv3(1000000, True)

Timing v1 Simulation on 1 million trials
661 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Timing v2 Simulation on 1 million trials
576 ms ± 5.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Timing v3 Simulation on 1 million trials
26.4 ms ± 897 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Woah...A Numpy only implementation is almost 10 times faster than my v1 and v2 versions, although all three versions generate their own arrays. Lets modify all three function to accept external arrays so that we can compare the results as well

In [14]:
trials = 1000000
Winning_Door = np.random.randint(1,4,trials)
Selected_Door = np.random.randint(1,4,trials)

def MontySimulation(trials,Winning_Door,Selected_Door, supress=False):
    if (supress == False):
        print("Simulating ",trials," trials")
    Result = 0

    #Running the trial where the contestant never switches
    for i in range(0, len(Winning_Door)):
        Result += MontyTrial(Winning_Door[i], Selected_Door[i], False)

    Result = Result/trials
    if (supress == False):
        print("Probability of winning with no switching: ", Result)
        print("Probability of winning with always switching: ", 1 - Result)

def MontySimulationv2(trials,Winning_Door,Selected_Door, supress=False):
    if (supress == False):
        print("Simulating ",trials," trials in v2")
    Result = 0

    #Running the trial where the contestant never switches
    for i in range(0, len(Winning_Door)):
        Result += MontyTrialv2(Winning_Door[i], Selected_Door[i], False)

    Result = Result/trials
    if (supress == False):
        print("Probability of winning with no switching: ", Result)
        print("Probability of winning with always switching: ", 1 - Result)

def MontySimulationv3(trials,Winning_Door,Selected_Door, supress=False):
    if (supress == False):
        print("Simulating ",trials," trials in v3 (Numpy Version)")
    #Generate a winning and a trial array and compare. A true in this array results in a win in case of no switching
    Result = Winning_Door == Selected_Door
    
    #If Winning door and the selected doors are the same and there is no switching, then its a win
    Noswitch_win = Result.sum()/trials
    
    if (supress == False):
        print("Probability of winning with no switching: ", Noswitch_win)
        #Winning with switching is 1 - probability of winning without switching
        print("Probability of winning with switching: ", 1 - Noswitch_win)

In [15]:
MontySimulation(trials, Winning_Door, Selected_Door)
MontySimulationv2(trials, Winning_Door, Selected_Door)
MontySimulationv3(trials, Winning_Door, Selected_Door)

Simulating  1000000  trials
Probability of winning with no switching:  0.333612
Probability of winning with always switching:  0.666388
Simulating  1000000  trials in v2
Probability of winning with no switching:  0.333612
Probability of winning with always switching:  0.666388
Simulating  1000000  trials in v3 (Numpy Version)
Probability of winning with no switching:  0.333612
Probability of winning with switching:  0.666388


Results from all three versions are the same. Lets do a final timing run for 10 million trials

In [16]:
trials = 10000000
Winning_Door = np.random.randint(1,4,trials)
Selected_Door = np.random.randint(1,4,trials)
print("Timing v1 Simulation on 10 million trials")
%timeit MontySimulation(trials, Winning_Door, Selected_Door, True)
print("Timing v2 Simulation on 10 million trials")
%timeit MontySimulationv2(trials, Winning_Door, Selected_Door, True)
print("Timing v3 Simulation on 10 million trials")
%timeit MontySimulationv3(trials, Winning_Door, Selected_Door, True)

Timing v1 Simulation on 10 million trials
6.68 s ± 395 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Timing v2 Simulation on 10 million trials
5.73 s ± 498 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Timing v3 Simulation on 10 million trials
38.6 ms ± 17.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


The Numpy implementation is now even faster pointing to the fact that generating the random array was the most time consuming part in the v3 algorithm.

### Rounding Up

This notebook was my first attempt at implementing a problem and documenting the findings. Along the way I realsied there was further opportunity for me to optimize my code by optimizing my logic and also using faster tools (Numpy)

About the monty hall problem:

- It is a well documented problem hence the results were not surprising
- This is a significant deviation from the mean with a smaller number of trials, however the values converge for higher number of trials (which is to be expected)
